## Apartado 1

In [5]:
import socket 
import sys

# Create a TCP/IP socket
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_address = ('localhost', 6780)
sock.bind (server_address)
sock.listen (1)
connection, client_address = sock.accept()
data = ""
while True:
    data = connection.recv(1024) .decode()
    print("RX: ",data)
    if data == "EXIT":
        connection.sendall("BYE!".encode())
        break
    connection.sendall("Echo".encode())

connection.close()

RX:  asdfj
RX:  EXIT


## Aaprtado 2

In [1]:
MTU = 20

In [3]:
import socket 
import sys

# Create a TCP/IP socket
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_address = ('localhost', 6782)
sock.bind (server_address)
sock.listen ()
print("accept")
connection, client_address = sock.accept()
data = ""
all_data = ""
count = 0
while True:
    data = connection.recv(20).decode()
    if (len(data) == 20):
        all_data += data
        count += 1
        if (count %10 == 0):
            print(count)
    else:
        print("RX: ",data)
        break
    
connection.sendall("Bye".encode())
connection.close()

accept
esperando
esperando
esperando
esperando
esperando
RX:  s


## Apartado 3

In [1]:
import socket
import os

# Registro de archivos bloqueados
locked_files = {}

def handle_get(connection, filename):
    if os.path.exists(filename):
        with open(filename, "rb") as file:
            connection.sendall(file.read())
    else:
        connection.sendall(b"ERROR: Archivo no encontrado")

def handle_put(connection, filename):
    with open(filename, "wb") as file:
        while True:
            data = connection.recv(1024)
            if not data:
                break
            file.write(data)

def handle_lock(connection, filename, client_info):
    if filename in locked_files:
        connection.sendall(b"ERROR: Archivo ya bloqueado")
    else:
        locked_files[filename] = client_info
        connection.sendall(b"Archivo bloqueado con exito")

def handle_unlock(connection, filename, client_info):
    if locked_files.get(filename) == client_info:
        del locked_files[filename]
        connection.sendall(b"Archivo desbloqueado con exito")
    else:
        connection.sendall(b"ERROR: No tienes permiso para desbloquear este archivo")

def handle_client(connection, client_address):
    try:
        while True:
            data = connection.recv(1024).decode()
            if data:
                command, filename = data.split(' ', 1)
                if command == 'get':
                    handle_get(connection, filename)
                elif command == 'put':
                    handle_put(connection, filename)
                elif command == 'lock':
                    handle_lock(connection, filename, client_address)
                elif command == 'unlock':
                    handle_unlock(connection, filename, client_address)
            else:
                break
    finally:
        connection.close()

def start_server(address, port):
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server_address = (address, port)
    sock.bind(server_address)
    sock.listen(1)

    while True:
        connection, client_address = sock.accept()
        handle_client(connection, client_address)

start_server('localhost', 6782)


KeyboardInterrupt: 